In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 28768697
paper_name = 'zacchi_bernstein_2017' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/TableS1-S6.xlsx', sheet_name='ST-1', skiprows=10)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 2851 x 11


In [7]:
original_data.head()

,Plate,Row,Column,Original Signal Value,Normalized Value,Log2 Normalized value,Z-Scores,P-Value,ORF,Gene,Description
0,[4],I,5,45902.86,3.454527,1.788488,6.605890,3.951400e-11,YKL008C,LAC1,Ceramide synthase component; involved in synth...
1,[3],M,5,77489.63,3.341991,1.740708,6.294210,3.089800e-10,YER065C,ICL1,Isocitrate lyase; catalyzes the formation of s...
2,[1],J,11,71690.99,3.304119,1.724266,6.189318,6.042400e-10,YLR073C,RFU1,Protein that inhibits Doa4p deubiquitinating a...
3,[2],K,11,41766.77,3.125455,1.644066,5.694488,1.237420e-08,YPL201C,YIG1,Protein that interacts with glycerol 3-phospha...
4,[2],F,9,55963.26,3.117048,1.640180,5.671203,1.417980e-08,YOR049C,RSB1,Suppressor of sphingoid LCB sensitivity of an ...


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Plate, Row, Column, Original Signal Value, Normalized Value, Log2 Normalized value, Z-Scores, P-Value, ORF, Gene, Description]
Index: []


In [12]:
original_data['data'] = original_data['Log2 Normalized value']

In [13]:
original_data.set_index('ORF', inplace=True)
original_data.index.name='orf'

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(2826, 7)

# Prepare the final dataset

In [16]:
data = original_data[['data']].copy()

In [17]:
dataset_ids = [16445]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,16445
data_type,value
orf,
YAL002W,-0.945618
YAL004W,0.675341
YAL007C,-1.059152
YAL008W,-1.276775
YAL009W,0.624556


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 10


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [22]:
data.head()

,dataset_id,16445
,data_type,value
gene_id,orf,
2,YAL002W,-0.945618
1863,YAL004W,0.675341
5,YAL007C,-1.059152
6,YAL008W,-1.276775
7,YAL009W,0.624556


In [23]:
data.shape

(2816, 1)

# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [27]:
data_all.head()

dataset_id          16445          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.945618 -2.020784
1863    YAL004W  0.675341  1.436412
5       YAL007C -1.059152 -2.262931
6       YAL008W -1.276775 -2.727078
7       YAL009W  0.624556  1.328097

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 28768697...
Inserting the new data...


100%|██████████| 1/1 [00:04<00:00,  4.26s/it]

Updating the data_modified_on field...
